# Crop Seed Segmentation Guide for Sorghum Seed

A lot of the codes are adapted from Toda et al. 2020
Their github: https://github.com/totti0223/crop_seed_instance_segmentation.git

Download the models from their github page, I provided the link here just in case:
The folder is named "data".
Link: https://drive.google.com/file/d/1g8bg9ter9DlKWgs0lfPZMQemRlzRVOQr/view?usp=sharing

# Let's import all the modules needed for this notebook: 

1. warnings
2. scikit-image
3. sys
4. numpy
5. pathlib
6. opencv
7. pandas
8. tensorflow >= 2.10.0
9. keras
10. os
11. pytesseract
12. matplotlib


In [3]:
import warnings
warnings.filterwarnings('ignore')  #general warning suppression
import skimage.io
import sys
import numpy as np
import json
from pathlib import Path
import cv2 as cv ##opencv
from skimage.measure import label, regionprops, regionprops_table
import pandas as pd
import tensorflow as tf 
import keras.backend as K 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  #tensorflow deprecration warning suppression
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
import pytesseract

tf.__version__

2024-05-28 11:00:44.417249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'2.10.0'

In [5]:
if not os.path.exists("Mask-RCNN"):
    !git clone https://github.com/NikeeShrestha/Mask-RCNN.git ##this is adapted for tensorflow 2.10.0
    print("Cloned repository")
else:
    print("Mask_RCNN directory already exists")

Mask_RCNN directory already exists


In [6]:
sys.path.append("Mask-RCNN") 

from mrcnn.visualize import display_images
from mrcnn.model import log
import mrcnn.model as modellib
from mrcnn.config import Config
from mrcnn import utils, visualize
print("done")

done


In [7]:
class InferenceConfig(Config):  ##Config is the modellib fucntion of Mask-RCNN
    NAME = "seed" ##you can name it anything
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1  # background + 1 seeds
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 8192
    DETECTION_MAX_INSTANCES = 1000
    IMAGE_RESIZE_MODE = "pad64"
    RPN_NMS_THRESHOLD = 0.4
    DETECTION_MIN_CONFIDENCE = 0

config = InferenceConfig()

# Load the model with specific directory when model is saved

In [8]:
K.clear_session()
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=".") 
##mode=inference means you want to use pre-trained model and predict on the image
model.load_weights("data/other/model_weights/rice.h5", by_name=True) ##I am using rice model but if you want, you can use wheat model as well. It won't segment each seed in the image though.
print("model weights loaded")

Instructions for updating:
box_ind is deprecated, use box_indices instead
Instructions for updating:
Use fn_output_signature instead
model weights loaded


# Segmentation and Data extraction; Length, Width, Area and three Color Channels

Cropping image should be customized based on the dimension of your image

In [19]:
path_directory="rawimages/Tutorial/"
Processed_image_directory="rawimages/processed/"
path1=Path(path_directory) ## Directory where your raw images area 
path=path1.glob("*.jpg") ##My images are in "jpg" format

MorphologicalData=[]
for i in path:
    img=cv.imread(str(i))##converting i into string with str function
    seed_tag=img[2800:3500,0:800] ##Cropping the seed tag
    thresh, seed_bw = cv.threshold(seed_tag, 155, 255, cv.THRESH_BINARY) ##Change to binary
    print(i)
    text=pytesseract.image_to_string(seed_bw ,config='--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789') ##recognize the text
    text=text.strip() ##remove /n from text
    print(text) ##print plotID
    seed_grain=img[1410:2610,645:1815] 
    seed_grain=cv.GaussianBlur(seed_grain,(5,5),0) 
    seed_gray=cv.cvtColor(seed_grain, cv.COLOR_BGR2GRAY) ##convert to grayscale
    thresh, img_bw = cv.threshold(seed_gray, 45, 255, cv.THRESH_BINARY) ##convert to binary with anything vale of 65 changes to white color
    masked= cv.bitwise_and(seed_grain, seed_grain, mask=img_bw) ##Masking together 
    cv.imwrite(Processed_image_directory + str(text) + "_" + os.path.basename(str(i)), masked) ##saving the picture
    
    results = model.detect([masked], verbose=1)
    r = results[0] 
    mask1=r['masks']
    visualize.display_instances(
            masked, r['rois'], r['masks'], r['class_ids'],
            ["",""], ["" for x in range(len(r['scores']))],
            show_bbox=True, show_mask=True,
            title="")
    
    ##Region Properties
    temp_morphologydata=[]

    for i in range(r['rois'].shape[0]):
        singlemaskpred=r['masks'][:,:,i]
        props=skimage.measure.regionprops(singlemaskpred.astype(int))[0]
        area=props.area
        major_axis_length=props.major_axis_length
        minor_axis_length=props.minor_axis_length
        temp_morphologydata.append({
            'area':area,
            'major_axis_length':major_axis_length,
            'minor_axis_length':minor_axis_length
        })
        # count=count+1
    df=pd.DataFrame(temp_morphologydata)
    df = df.drop(df['area'].idxmax()) ##Dropping the maximum data of area because sometimes huge masks are created and we do not want to consider it while taking the average.
    df = df.drop(df['area'].idxmin()) ##Dropping the minimum data of area because sometimes small particles can be segmented.
    
    ##Parameters that we measure
    Label=mask1.shape[-1]
    imagename=os.path.basename(str(i))
    Area=df.mean()['area']
    Length=df.mean()['major_axis_length']
    Width=df.mean()['minor_axis_length']
    # EquivalentDiameter=df.mean()['equivalent_diameter']
    
    ##RGB extraction
    B=[]
    G=[]
    R=[]
    for i in range(r['masks'].shape[-1]):
        mask = r['masks'][:, :, i]
        color=pd.DataFrame(seed_grain[mask])
        B.append(color.mean()[0])
        G.append(color.mean()[1])
        R.append(color.mean()[2])
    print("done")
    AverageB=sum(B)/len(B)
    AverageG=sum(G)/len(G)
    AverageR=sum(R)/len(R)
    
    #creating dictionary ##You can edit it based on what you want
    MorphologicalData.append(
        {
            'imagename': imagename,
            'PlotID': text,
            'Count': Label,
            'Area': Area,
            'Length': Length,
            'Width': Width,
            'AverageB':AverageB,
            'AverageG':AverageG,
            'AverageR':AverageR  
        }
    )
  
MorphologicalData=pd.DataFrame(MorphologicalData)
MorphologicalData.to_csv(path_directory + "MorphologicalData.csv") ##Saving the data extracted in csv file